# Neural Collaborative Filtering for Rating Prediction

## Motivation

The original NCF (He et al., 2017) was designed for **implicit feedback** (binary interactions). 

For **explicit feedback** (ratings 1-5), we modify:
1. **Loss**: BCE → MSE (regression task)
2. **Output**: Scaled sigmoid to [1, 5]
3. **Data**: Actual ratings, no negative sampling

## Models to Compare

| Model | Description |
|-------|-------------|
| **GMF** | Generalized Matrix Factorization (linear) |
| **MLP** | Multi-Layer Perceptron (non-linear) |
| **NeuMF-end** | GMF + MLP trained end-to-end |
| **NeuMF-pre** | GMF + MLP with pre-trained initialization |

## Step 1: Setup and Imports

In [ ]:
import os
import sys
import time
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Add src to path
current_dir = os.getcwd()
path = current_dir
while True:
    if os.path.basename(path) == "src":
        if path not in sys.path:
            sys.path.insert(0, path)
        break
    parent = os.path.dirname(path)
    if parent == path:
        break
    path = parent

from helpers import download_ml1m_dataset

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Device: {device}")

## Step 2: Hyperparameters

In [ ]:
# Paths
DATA_DIR = os.path.join(os.path.dirname(os.getcwd()), '..', 'data')
MODEL_PATH = os.path.join(os.path.dirname(os.getcwd()), '..', 'models')
os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(MODEL_PATH, exist_ok=True)

# Hyperparameters
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.2
BATCH_SIZE = 256
EPOCHS = 30
FACTOR_NUM = 32
NUM_LAYERS = 3
EARLY_STOPPING_PATIENCE = 5
TEST_RATIO = 0.2

RATING_MIN = 1.0
RATING_MAX = 5.0

## Step 3: Load Data

In [ ]:
ratings_file = download_ml1m_dataset(DATA_DIR)

ratings_df = pd.read_csv(
    ratings_file,
    sep='::',
    engine='python',
    names=['user_id', 'item_id', 'rating', 'timestamp'],
    encoding='latin-1'
)

# Re-index
user_ids = ratings_df['user_id'].unique()
item_ids = ratings_df['item_id'].unique()
user_to_idx = {uid: idx for idx, uid in enumerate(user_ids)}
item_to_idx = {iid: idx for idx, iid in enumerate(item_ids)}
ratings_df['user_idx'] = ratings_df['user_id'].map(user_to_idx)
ratings_df['item_idx'] = ratings_df['item_id'].map(item_to_idx)

num_users = len(user_ids)
num_items = len(item_ids)

print(f"Users: {num_users}, Items: {num_items}, Ratings: {len(ratings_df):,}")

In [ ]:
# Train/Test split
train_df, test_df = train_test_split(ratings_df, test_size=TEST_RATIO, random_state=42)
print(f"Train: {len(train_df):,}, Test: {len(test_df):,}")

## Step 4: Dataset Class

In [ ]:
class RatingDataset(data.Dataset):
    """Dataset for explicit rating prediction (no negative sampling)."""
    
    def __init__(self, df):
        self.users = torch.LongTensor(df['user_idx'].values)
        self.items = torch.LongTensor(df['item_idx'].values)
        self.ratings = torch.FloatTensor(df['rating'].values)
    
    def __len__(self):
        return len(self.ratings)
    
    def __getitem__(self, idx):
        return self.users[idx], self.items[idx], self.ratings[idx]


train_dataset = RatingDataset(train_df)
test_dataset = RatingDataset(test_df)
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

## Step 5: NCF Model for Rating Prediction

In [ ]:
class NCFRating(nn.Module):
    """
    NCF model adapted for rating prediction.
    
    Supports: 'GMF', 'MLP', 'NeuMF-end', 'NeuMF-pre'
    Output: Scaled sigmoid → [rating_min, rating_max]
    """
    
    def __init__(self, num_users, num_items, factor_num, num_layers,
                 dropout, model_name='NeuMF-end', rating_min=1.0, rating_max=5.0,
                 GMF_model=None, MLP_model=None):
        super(NCFRating, self).__init__()
        
        self.model_name = model_name
        self.dropout = dropout
        self.rating_min = rating_min
        self.rating_max = rating_max
        self.GMF_model = GMF_model
        self.MLP_model = MLP_model
        
        # GMF Embeddings
        if model_name != 'MLP':
            self.embed_user_GMF = nn.Embedding(num_users, factor_num)
            self.embed_item_GMF = nn.Embedding(num_items, factor_num)
        
        # MLP Embeddings
        if model_name != 'GMF':
            mlp_embed_dim = factor_num * (2 ** (num_layers - 1))
            self.embed_user_MLP = nn.Embedding(num_users, mlp_embed_dim)
            self.embed_item_MLP = nn.Embedding(num_items, mlp_embed_dim)
            
            # MLP Layers
            layers = []
            for i in range(num_layers):
                input_size = factor_num * (2 ** (num_layers - i))
                layers.append(nn.Dropout(p=dropout))
                layers.append(nn.Linear(input_size, input_size // 2))
                layers.append(nn.ReLU())
            self.MLP_layers = nn.Sequential(*layers)
        
        # Prediction Layer
        if model_name in ['MLP', 'GMF']:
            predict_size = factor_num
        else:
            predict_size = factor_num * 2
        
        self.predict_layer = nn.Linear(predict_size, 1)
        self._init_weights()
    
    def _init_weights(self):
        if self.model_name != 'NeuMF-pre':
            # Random initialization
            for name, param in self.named_parameters():
                if 'embed' in name:
                    nn.init.normal_(param, std=0.01)
                elif 'weight' in name:
                    nn.init.xavier_uniform_(param)
                elif 'bias' in name:
                    nn.init.zeros_(param)
        else:
            # Pre-trained initialization from GMF and MLP
            self.embed_user_GMF.weight.data.copy_(self.GMF_model.embed_user_GMF.weight)
            self.embed_item_GMF.weight.data.copy_(self.GMF_model.embed_item_GMF.weight)
            self.embed_user_MLP.weight.data.copy_(self.MLP_model.embed_user_MLP.weight)
            self.embed_item_MLP.weight.data.copy_(self.MLP_model.embed_item_MLP.weight)
            
            for (m1, m2) in zip(self.MLP_layers, self.MLP_model.MLP_layers):
                if isinstance(m1, nn.Linear) and isinstance(m2, nn.Linear):
                    m1.weight.data.copy_(m2.weight)
                    m1.bias.data.copy_(m2.bias)
            
            # Combine prediction weights
            predict_weight = torch.cat([
                self.GMF_model.predict_layer.weight,
                self.MLP_model.predict_layer.weight
            ], dim=1)
            predict_bias = (self.GMF_model.predict_layer.bias + 
                           self.MLP_model.predict_layer.bias) / 2
            
            self.predict_layer.weight.data.copy_(0.5 * predict_weight)
            self.predict_layer.bias.data.copy_(predict_bias)
    
    def forward(self, user, item):
        # GMF path
        if self.model_name != 'MLP':
            user_gmf = self.embed_user_GMF(user)
            item_gmf = self.embed_item_GMF(item)
            output_gmf = user_gmf * item_gmf
        
        # MLP path
        if self.model_name != 'GMF':
            user_mlp = self.embed_user_MLP(user)
            item_mlp = self.embed_item_MLP(item)
            interaction = torch.cat([user_mlp, item_mlp], dim=-1)
            output_mlp = self.MLP_layers(interaction)
        
        # Combine
        if self.model_name == 'GMF':
            concat = output_gmf
        elif self.model_name == 'MLP':
            concat = output_mlp
        else:
            concat = torch.cat([output_gmf, output_mlp], dim=-1)
        
        # Predict and scale to rating range
        logits = self.predict_layer(concat).view(-1)
        rating_range = self.rating_max - self.rating_min
        return torch.sigmoid(logits) * rating_range + self.rating_min

## Step 6: Evaluation and Training Functions

In [ ]:
def evaluate(model, data_loader, device):
    """Compute RMSE and MAE."""
    model.eval()
    preds, actuals = [], []
    
    with torch.no_grad():
        for user, item, rating in data_loader:
            user, item = user.to(device), item.to(device)
            pred = model(user, item)
            preds.extend(pred.cpu().numpy())
            actuals.extend(rating.numpy())
    
    preds, actuals = np.array(preds), np.array(actuals)
    rmse = np.sqrt(np.mean((preds - actuals) ** 2))
    mae = np.mean(np.abs(preds - actuals))
    return rmse, mae


def train_model(model, model_name, train_loader, test_loader, 
                epochs, lr, patience, device, save_path):
    """Train a single model and return history."""
    model = model.to(device)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    
    best_rmse = float('inf')
    patience_counter = 0
    history = {'train_loss': [], 'test_rmse': [], 'test_mae': []}
    
    print(f"\n{'='*60}")
    print(f"Training {model_name}")
    print(f"{'='*60}")
    
    for epoch in range(epochs):
        model.train()
        total_loss = 0
        start = time.time()
        
        for user, item, rating in train_loader:
            user, item, rating = user.to(device), item.to(device), rating.to(device)
            optimizer.zero_grad()
            pred = model(user, item)
            loss = criterion(pred, rating)
            loss.backward()
            optimizer.step()
            total_loss += loss.item() * len(rating)
        
        avg_loss = total_loss / len(train_loader.dataset)
        test_rmse, test_mae = evaluate(model, test_loader, device)
        elapsed = time.time() - start
        
        history['train_loss'].append(avg_loss)
        history['test_rmse'].append(test_rmse)
        history['test_mae'].append(test_mae)
        
        if test_rmse < best_rmse:
            best_rmse = test_rmse
            patience_counter = 0
            torch.save(model.state_dict(), save_path)
            marker = " *"
        else:
            patience_counter += 1
            marker = ""
        
        print(f"Epoch {epoch+1:02d} | Loss: {avg_loss:.4f} | RMSE: {test_rmse:.4f} | "
              f"MAE: {test_mae:.4f} | {elapsed:.1f}s{marker}")
        
        if patience_counter >= patience:
            print(f"Early stopping at epoch {epoch+1}")
            break
    
    # Load best weights
    model.load_state_dict(torch.load(save_path))
    return model, history, best_rmse

---
## Step 7: Train GMF (Generalized Matrix Factorization)

**GMF** uses element-wise product of user and item embeddings (linear interaction).

In [ ]:
gmf_model = NCFRating(
    num_users=num_users, num_items=num_items,
    factor_num=FACTOR_NUM, num_layers=NUM_LAYERS,
    dropout=DROPOUT_RATE, model_name='GMF',
    rating_min=RATING_MIN, rating_max=RATING_MAX
)

gmf_model, gmf_history, gmf_best = train_model(
    gmf_model, 'GMF', train_loader, test_loader,
    EPOCHS, LEARNING_RATE, EARLY_STOPPING_PATIENCE, device,
    os.path.join(MODEL_PATH, 'GMF-Rating.pth')
)

---
## Step 8: Train MLP (Multi-Layer Perceptron)

**MLP** uses a deep neural network for non-linear user-item interactions.

In [ ]:
mlp_model = NCFRating(
    num_users=num_users, num_items=num_items,
    factor_num=FACTOR_NUM, num_layers=NUM_LAYERS,
    dropout=DROPOUT_RATE, model_name='MLP',
    rating_min=RATING_MIN, rating_max=RATING_MAX
)

mlp_model, mlp_history, mlp_best = train_model(
    mlp_model, 'MLP', train_loader, test_loader,
    EPOCHS, LEARNING_RATE, EARLY_STOPPING_PATIENCE, device,
    os.path.join(MODEL_PATH, 'MLP-Rating.pth')
)

---
## Step 9: Train NeuMF-end (End-to-End)

**NeuMF-end** combines GMF and MLP paths, trained from scratch.

In [ ]:
neumf_end_model = NCFRating(
    num_users=num_users, num_items=num_items,
    factor_num=FACTOR_NUM, num_layers=NUM_LAYERS,
    dropout=DROPOUT_RATE, model_name='NeuMF-end',
    rating_min=RATING_MIN, rating_max=RATING_MAX
)

neumf_end_model, neumf_end_history, neumf_end_best = train_model(
    neumf_end_model, 'NeuMF-end', train_loader, test_loader,
    EPOCHS, LEARNING_RATE, EARLY_STOPPING_PATIENCE, device,
    os.path.join(MODEL_PATH, 'NeuMF-end-Rating.pth')
)

---
## Step 10: Train NeuMF-pre (Pre-trained Initialization)

**NeuMF-pre** initializes GMF and MLP paths with pre-trained weights, then fine-tunes.

In [ ]:
neumf_pre_model = NCFRating(
    num_users=num_users, num_items=num_items,
    factor_num=FACTOR_NUM, num_layers=NUM_LAYERS,
    dropout=DROPOUT_RATE, model_name='NeuMF-pre',
    rating_min=RATING_MIN, rating_max=RATING_MAX,
    GMF_model=gmf_model,  # Pre-trained GMF
    MLP_model=mlp_model   # Pre-trained MLP
)

neumf_pre_model, neumf_pre_history, neumf_pre_best = train_model(
    neumf_pre_model, 'NeuMF-pre', train_loader, test_loader,
    EPOCHS, LEARNING_RATE, EARLY_STOPPING_PATIENCE, device,
    os.path.join(MODEL_PATH, 'NeuMF-pre-Rating.pth')
)

---
## Step 11: Comparative Results

In [ ]:
# Summary table
results = {
    'GMF': gmf_best,
    'MLP': mlp_best,
    'NeuMF-end': neumf_end_best,
    'NeuMF-pre': neumf_pre_best
}

print("\n" + "="*50)
print("COMPARATIVE RESULTS (Best RMSE)")
print("="*50)
for name, rmse in results.items():
    print(f"{name:12s} | RMSE: {rmse:.4f}")
print("="*50)

best_model = min(results, key=results.get)
print(f"\nBest Model: {best_model} (RMSE: {results[best_model]:.4f})")

In [ ]:
# Plot training curves
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

histories = [
    ('GMF', gmf_history),
    ('MLP', mlp_history),
    ('NeuMF-end', neumf_end_history),
    ('NeuMF-pre', neumf_pre_history)
]

for ax, (name, hist) in zip(axes.flat, histories):
    ax.plot(hist['test_rmse'], 'r-', label='RMSE')
    ax.plot(hist['test_mae'], 'b-', label='MAE')
    ax.set_title(f'{name}')
    ax.set_xlabel('Epoch')
    ax.set_ylabel('Error')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(MODEL_PATH, 'ncf_rating_comparison.png'), dpi=150)
plt.show()

## Summary

### Key Modifications for Rating Prediction

| Original NCF | Rating NCF |
|--------------|------------|
| BCEWithLogitsLoss | MSELoss |
| Binary labels (0/1) | Actual ratings (1-5) |
| Negative sampling | No negative sampling |
| HR@K, NDCG@K | RMSE, MAE |

### References

- He, X., et al. (2017). Neural Collaborative Filtering. WWW'17.